In [1]:
from pathlib import Path
from time import time
from collections import deque
from random import sample

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

import gym
from gym.envs.registration import register

In [2]:
results_path = Path('results', 'trading_bot')

## Helper Functions

In [3]:
def format_time(t):
    m_, s = divmod(t, 60)
    h, m = divmod(m_, 60)
    return '{:02.0f}:{:02.0f}:{:02.0f}'.format(h, m, s)

## Setup Environment

In [5]:
# Set trading days to 1 year
trading_days = 252

In [6]:
register(
    id='trading-v0',
    entry_point='trading_env:TradingEnvironment',
    max_episode_steps=trading_days
)

In [23]:
# Setting trading costs to 0 for now
trading_cost_bps = 0
time_cost_bps = 0

In [24]:
trading_environment = gym.make('trading-v0')
trading_environment.env.trading_days = trading_days
trading_environment.env.trading_cost_bps = trading_cost_bps
trading_environment.env.time_cost_bps = time_cost_bps
trading_environment.env.ticker = 'AAPL'

INFO:trading_env:loading data for AAPL...
INFO:trading_env:got data for AAPL...
INFO:trading_env:None


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 9367 entries, (Timestamp('1981-01-30 00:00:00'), 'AAPL') to (Timestamp('2018-03-27 00:00:00'), 'AAPL')
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   returns  9367 non-null   float64
 1   ret_2    9367 non-null   float64
 2   ret_5    9367 non-null   float64
 3   ret_10   9367 non-null   float64
 4   ret_21   9367 non-null   float64
 5   rsi      9367 non-null   float64
 6   macd     9367 non-null   float64
 7   atr      9367 non-null   float64
 8   stoch    9367 non-null   float64
 9   ultosc   9367 non-null   float64
dtypes: float64(10)
memory usage: 1.5+ MB


C:\Users\jimmy.liang\Anaconda3\envs\finalProject\lib\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [25]:
trading_environment.observation_space

Box(returns    -0.518692
ret_2     -13.186786
ret_5      -9.157841
ret_10     -6.979122
ret_21     -5.289787
rsi        -1.529044
macd       -5.407722
atr        -0.615589
stoch      -2.762308
ultosc     -3.964109
dtype: float32, returns     0.332152
ret_2      11.431712
ret_5      10.235379
ret_10      9.135829
ret_21      8.238228
rsi         1.499695
macd        5.705033
atr         5.415272
stoch       2.712635
ultosc      2.763141
dtype: float32, (10,), float32)

In [18]:
trading_environment.action_space

Discrete(3)

In [16]:
trading_environment.spec.max_episode_steps

252

In [32]:
trading_environment.reset()
trading_environment.step(2)

(array([-0.01958736, -0.35493985,  2.07697324,  0.12507575,  0.96113906,
         1.22884202, -0.09630326, -0.3272639 , -0.11533735,  0.4013755 ]),
 -0.001,
 False,
 {'reward': -0.001, 'nav': 1.0, 'costs': 0.001})